## Working with a Landsat image

### Download your Landsat scene

Go to https://libra.developmentseed.org/ and browse for a landsat image. Find images over a __US city__ and select a relativly non-cloudy scene in the left list.

![Libra browser](img/libra-screenshot.png)

Once you have a scene you are happy with, click "Download Bands". The modal seen below should pop up. You must download the bands individually; download the the Red, Green, and Blue bands.

### Move the Landsat scene GeoTiffs to a readable location

The filenames should be something like:

```
LC08_L1TP_014032_20171018_20171025_01_T1_B2.TIF
LC08_L1TP_014032_20171018_20171025_01_T1_B3.TIF
LC08_L1TP_014032_20171018_20171025_01_T1_B4.TIF
```

Each of these files is a GeoTiff of one band from the Landsat scene. Move the Landsat scene GeoTiffs to the `data` directory of your clone of the repository. Replace the `<LANDSAT GEOTIFF NAME>` text below with the  Landsat scene GeoTiff file, minus the band suffix; for example, `LC08_L1TP_014032_20171018_20171025_01_T1`.

In [ ]:
import os

landsat_scene_name = "LC08_L1TP_027039_20171029_20171109_01_T1"

red_path = os.path.join("/home/hadoop/data/", 
                        "{}_B4.TIF".format(landsat_scene_name))
green_path = os.path.join("/home/hadoop/data/", 
                       "{}_B3.TIF".format(landsat_scene_name))
blue_path = os.path.join("/home/hadoop/data/", 
                       "{}_B2.TIF".format(landsat_scene_name))

We'll be using [rasterio](https://github.com/mapbox/rasterio) to read the raster data in. When you open a raster file in rasterio you get back a `DatasetReader`, which contains ways to access the raster data as a [numpy](http://www.numpy.org/) array, as well as metadata such as the coordinate reference system and the raster's georeferenced bounds.

For now we'll directly call `ds.read()` to read the data as a numpy array, without capturing any other metadata from the `DatasetReader`.

In [ ]:
import rasterio

with rasterio.open(red_path) as ds:
    red_data = ds.read()

`red_data` is a numpy `ndarray` containing the raster values of the red band of our Landsat scene. An `ndarray` is an N-Dimensional Array, which can be thought of as an N dimensional matrix. If we look at the "shape" of the array, we can see that it has 3 dimensions:

In [ ]:
red_data.shape

`shape` returns a tuple that shows the size of each dimension. This raster data has length 1 at the first dimension, because that dimension represents the band of the raster - this is a single band raster, so there is only 2D image in this ndarray. If we read in a 3 band raster, fo instance, this might read something like `(3, 761, 7611)`. The second and third lenghts are the number of rows and columns in the image, respectively.

## Visualizing the red band

One of the first things we'll want to do with raster data is visualize it. This is only a single band of Landsat, so at first we'll just create a grayscale image.

We'll be using [matplotlib](https://matplotlib.org/), which is a great library to use in combination with Jupyter notebooks in order to visualize your data. A lot of matplotlib is about creating charts and graphs, but it can also show images. This snippet of code will render our raster as grayscale. Notice we are indexing into the 3D array to pull out the 2D image that is the first (and only) band:

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(red_data[0], cmap='gray')
plt.show()

Notice this image is dark. That's because Landsat data is stored in the `uint16` data type, which has a range of values from `0 - 65535`. However, the sensor data captured has a range of values generally a lot smaller than that. 

We can see this if we use `matplotlib` to show the histogram of data. We utilize the `np.ravel` method to flatten out our data into a 1D array:

In [ ]:
flattened = np.ravel(red_data)
plt.hist(flattened, bins='auto')  
plt.title("Histogram with 'auto' bins")
plt.show()

Most of our data lives in the left hand part of this graph, though there is noisey data all along the scale. One way to mitigate this for our visualization is to "clamp" the values to some sensible min and max data: if values are below or above a given range, we set them to our desired min and max.

I spent the many hours I had on a plane trip from Pennsylvania to California tweaking an appropriate min and max value for displaying Landsat 8 data. [These values](https://github.com/geotrellis/geotrellis-landsat-emr-demo/blob/master/server/src/main/scala/demo/Render.scala#L24) generally work across Landsat scenes, and have become known as the "[lossyrob magic numbers](https://github.com/s22s/raster-frames/blob/9208d2a6c0c48b00e01bf29fa3f55de81ef02f03/core/src/main/scala/astraea/spark/rasterframes/util/MultibandRender.scala#L100)":

In [ ]:
# https://github.com/geotrellis/geotrellis-landsat-emr-demo/blob/master/server/src/main/scala/demo/Render.scala#L24
MIN_LANDSAT_RENDER_VALUE = 4000
MAX_LANDSAT_RENDER_VALUE = 15176

There are a couple of ways to clamp the image to these values, with serious performance considerations. `numpy` utilizes a lot of native code in C for very fast performance. This shows off Python's ability to tie into native code well, and become a front end for fast performance-tuned library code. However, if we want python code to execute in the processing of data happening in C, it needs to call back up to the Python interpretor _per value_, which ends up taking a lot of time.

All in all, the point is, don't do what's in `incorrect_clamp`: avoid looping over a numpy array like a regular list at all cost. I ran the below code on a landsat scene and did not return in a couple of minutes:

In [ ]:
red = red_data[0]

# DO NOT USE
def incorrect_clamp(data, min_value, max_value):
    clamped = np.zeros(data.shape)
    for row in range(0, red.shape[0]):
        for col in range(0, red.shape[1]):
            v = red[row][col]
            if v < min_value:
                v = 4000
            if v > max_value:
                v = 15176
            clamped_red[row][col] = v
    return clamped
            
# clamped_red = incorrect_clamp(data, MIN_LANDSAT_RENDER_VALUE, MAX_LANDSAT_RENDER_VALUE) # THIS RUNS SLOOOOWWWWWWW

The good news is, there's usually a way to do the thing you want to do directly using `numpy` capabilities. Here's a version of clamp that runs very fast:

In [ ]:
red = red_data[0]

# Correct: Clamp values to our range
def clamp(data, min_value, max_value):
    clamped = data.copy()
    clamped[clamped < min_value] = 4000
    clamped[clamped > max_value] = 15176
    return clamped

clamped_red = clamp(red, MIN_LANDSAT_RENDER_VALUE, MAX_LANDSAT_RENDER_VALUE)

Now when we render the image, we get something that looks a bit more sensible:

In [ ]:
plt.imshow(clamped_red, cmap='gray')
plt.show()

It's still hard to see because it's such a tiny image. Let's create a method that uses `subplots` to create a plot with a larger figure size. Creating a method is a great way to reuse code; we'll be using this one a lot in the code ahead:

In [ ]:
def show_image(img, cmap='gray'):
    fig, ax = plt.subplots(figsize=(16, 16))
    ax.imshow(img, cmap)
    plt.show()

In [ ]:
show_image(clamped_red, cmap='gray')

## Clipping to the city

The Landsat image is pretty big, so let's cut it down to make it more managable. This next set of code will download [GeoJSON](http://geojson.org/) from an open source repository on GitHub for a county. Figure out what state and county your city is in if you don't already know, and replace the values below:

In [ ]:
state_name, county_name = "TX", "Travis"

This next set of code will go fetch the GeoJSON for your chosen county, and print out the GeoJSON:

In [ ]:
def fetch_county_geojson(state, county):
    import urllib
    county_url = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/{}/{}.geo.json".format(state,county)
    return urllib.request.urlopen(county_url).read().decode("utf-8")
    
county_geojson = fetch_county_geojson(state_name, county_name)
print(county_geojson)

If you have trouble downloading the GeoJSON, got to [the GitHub repository we are pulling the data from](https://github.com/johan/world.geo.json/tree/master/countries/USA/) and find the correct state abbreviation and county name.

## Working with JSON

Python Standard Library's [json](https://docs.python.org/2/library/json.html) module can be used to encode and decode all JSON, including GeoJSON. Here we use the `loads` (load-string) method for loading the GeoJSON intoa Python `dict`.

In [ ]:
import json

js = json.loads(county_geojson)
js

Now that we have a `dict`, we can drill into it to get at the `geometry`.

In [ ]:
county_poly_json = js['features'][0]['geometry']

## Working with shapely

`shapely` is a library for working with vector data in Python. With `shapely`, we can turn our `dict`-encoded geometry GeoJSON into a `shapely` geometry:

In [ ]:
from shapely.geometry import shape

county_poly = shape(county_poly_json)
county_poly

Notice how Jupyter notebooks draw out a shapely geometry, which is a neat feature and great for getting a sense of your vector data.

## Reprojecting the geometry to the Landsat data CRS

As always, when working with geospatial data, you have to mind your [projections](https://en.wikipedia.org/wiki/Map_projection). The Coordinate Reference System (CRS) of the geometry is `WSG 84` (`EPSG:4326`), or "lat long" as it's commonly referred to. The Landsat data comes in a [UTM coordinate system](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system) projection, with the UTM zone based on where the Landsat scene was taken. So, in order to use our geometry to cut out data from our raster data, we'll need to reproject it.

First, we grab the `CRS` of the Landsat scene from the GeoTIFF:

In [ ]:
with rasterio.open(red_path) as ds:
    crs = ds.crs

Now we use a combination of `shapely` and a project called `pyproj` for the reprojection call. We'll wrap this in a function since it'll get a lot of use:

In [ ]:
# Reproject to LatLng
import pyproj
from functools import partial
from shapely.ops import transform

def reproject_geom(g, proj1, proj2):
    """
    Reprojects a shapely geometry
    """

    project = partial(
        pyproj.transform,
        proj1, # source coordinate system
        proj2) # destination coordinate system

    return transform(project, g)  # apply projection

In [ ]:
image_proj = pyproj.Proj(crs)
lat_lng_proj = pyproj.Proj(init="epsg:4326")

reprojected_county_poly = reproject_geom(county_poly, lat_lng_proj, image_proj)
reprojected_county_poly

Now that we have teh geometry in the correct projection, we can use the rasterio `mask` call to read in a cropped version of the data:

In [ ]:
from rasterio.mask import mask
from shapely.geometry import mapping

with rasterio.open(red_path) as ds:
    cropped_red_data, _ = mask(ds, [mapping(reprojected_county_poly)], crop=True)

In [ ]:
show_image(clamp(cropped_red_data[0],
           min_value=MIN_LANDSAT_RENDER_VALUE, 
           max_value=MAX_LANDSAT_RENDER_VALUE))

## Bringing in color

Let's add some color to the scene. We'll read in the Green and Blue bands:

In [ ]:
with rasterio.open(green_path) as ds:
    cropped_green_data, _ = mask(ds, [mapping(reprojected_county_poly)], crop=True)
    
with rasterio.open(blue_path) as ds:
    cropped_blue_data, _ = mask(ds, [mapping(reprojected_county_poly)], crop=True)

and then stack the data so that the numpy array we end up with represents a multiband RGB image.

In [ ]:
rgb = np.concatenate([cropped_red_data, cropped_green_data, cropped_green_data], axis=0)
rgb.shape

We can clamp the whole `ndarray` with our clamp method:

In [ ]:
rgb = clamp(rgb, min_value=MIN_LANDSAT_RENDER_VALUE, max_value=MAX_LANDSAT_RENDER_VALUE)

The result is an "rgb" image, but usually when people think of RGB values, they think of each of the channels - red, green, and blue - representing a byte value; these are the types of images that can have pixel values represented as hex values such as `#FFAF09`. Our `ndarray`, though, is `uint16`. We can "rescale" the pixel values in order to be bytes, to end up with a `uint8` RGB image:

In [ ]:
def rescale(data):
    """
    Rescales a uint16 Landsat band to uint8 values. Assumes the band is clamped.
    """
    old_range = MAX_LANDSAT_RENDER_VALUE - MIN_LANDSAT_RENDER_VALUE
    d = data.astype(float)
    rescaled = (((d - MIN_LANDSAT_RENDER_VALUE) * 255) / old_range)
    return rescaled.astype('uint8')
    
rgb_rescaled = rescale(rgb)

In order to show our multiband, RGB image encoded in our `ndarray`, we'll have to write another `show_*` method. This method will [transpose](https://en.wikipedia.org/wiki/Transpose) the ndarray to have each pixel value contain the 3 channels, instead of having each band in it's own column. In this way, the shape of the `ndarray` will move from `(3, X, Y)` to `(X, Y, 3)`:

In [ ]:
def show_rgb_image(img):
    fig, ax = plt.subplots(figsize=(16, 16))
    # Transpose the numpy array, to turn each pixel into a RGB pixel
    t = np.transpose(img, axes=[1, 2, 0])
    ax.imshow(t)
    plt.show()

In [ ]:
show_rgb_image(rgb_rescaled)

## Color correction with rio_color

Color correction via clamping and rescaling based on magic numbers is OK. But there's better tools for color correction of imagery, including [rio_color](https://github.com/mapbox/rio-color), from the makers of `rasterio`. `rio_color` provides a number of color correction methods, and we'll take a look at three of them:

In [ ]:
from rio_color.operations import (sigmoidal, 
                                  gamma, 
                                  simple_atmo)

### Refactoring our rescale method

In order to use the `rio_color` library, we need to rescale our image again, but instead of to bytes this time, we need floating point values between 0.0 and 1.0. Above we wrote a new method for showing RGB images, and we could take the same approach for our rescaling; instead, let's _refactor_ our method to be able to handle both the old case and the new case.

In [ ]:
def rescale(data,new_min, new_max,  dtype=None, old_min=None, old_max=None):
    """Rescales an array between data types and min/max values.

    Args:
        new_min: The target minimum value for the output array.                  
        new_max: The target maximum value for the output array. 
        dtype:   The target dtype for the output array. 
                 Defaults to the same type as the input array.
        old_min: The target minimum value for the output array. 
                 Defaults to the minimum value in data.
        old_max: The target maximum value for the output array. 
                 Defaults to the maximum value in data.

    Returns:
        The rescaled image.
    """
    if not dtype:
        dtype = data.dtype
    if new_min is None:
        new_min = np.iinfo(dtype).min
    if new_max is None:
        new_max = np.iinfo(dtype).max
    if old_min is None:         # Don't use `if old_min:`; 0 will pass conditional
        old_min = np.min(data)
    if old_max is None:
        old_max = np.max(data)
        
    old_range = old_max - old_min
    new_range = new_max - new_min
    d = clamp(data, old_min, old_max).astype('float64')
    rescaled = ((((d - old_min) * new_range) / old_range) + new_min)

    return rescaled.astype(dtype)

In [ ]:
rgb_rescaled = rescale(rgb, 
                       dtype=np.float, 
                       new_min = 0.0,
                       new_max = 1.0)

One handy method in python is the `help` method, which allows you to print out the PyDoc strings that describe the method. For instance, if I tried it on the function we just wrote, I'd see the documentation I wrote into that first string right after I defined the function:

In [ ]:
help(rescale)

More immediately useful, it can explain how to use the `rio_color` functions we just imported:

In [ ]:
help(sigmoidal)

Here I make a set of modifications with some sensible parameters to each method. Try reordering them and changing values to see if you can come up with a nicer snapshot of the city!

In [ ]:
x = rgb_rescaled
x = simple_atmo(x, 0.03, 3, 0.5)
x = gamma(x, 1.1)
x = sigmoidal(x, 3, 0.16)

rgb_color_corrected = rescale(x, dtype='uint8', new_min=0, new_max=255)

In [ ]:
show_rgb_image(rgb_color_corrected)

Now that we've put the time into correcting our image, let's go ahead and save it to disk for perpetuity. We'll use the [imageio]() library to write out a PNG. Like `matplotlib`, `imageio` requires the the dimensions to be `(ROWS, COLS, BANDS)` instead of `(BANDS, ROWS, COLS)`, so we'll have to transpose the `ndarray` again:

In [ ]:
import imageio

img = np.transpose(rgb_color_corrected, axes=[1, 2, 0])
imageio.imwrite("/home/hadoop/data/city.png", img)

You should find your image in the `data` directory of your repository clone.